In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime as dt
from datetime import datetime
from scipy.stats import linregress
from citipy import citipy



In [3]:
visitor_path = "Data/changes-visitors-covid.csv"
covid_path = "Data/covid-data.csv"
debt_path = "Data/debt-relief-covid.csv"
income_path = "Data/income-support-covid.csv"
face_path = "Data/face-covering-policies-covid.csv"
travel_path = "Data/international-travel-covid.csv"
school_path = "Data/school-closures-covid.csv"


In [4]:
visitor_data = pd.read_csv(visitor_path)
covid_data = pd.read_csv(covid_path)
debt_data = pd.read_csv(debt_path)
income_data = pd.read_csv(income_path)
face_data = pd.read_csv(face_path)
travel_data = pd.read_csv(travel_path)
school_data = pd.read_csv(school_path)


In [5]:
visitor_data.head()

,Entity,Code,Day,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces
0,Afghanistan,AFG,2020-02-17,0.00,4.000,1.333,5.667,1.000,-6.333
1,Afghanistan,AFG,2020-02-18,0.75,4.250,0.750,6.500,1.500,-3.250
2,Afghanistan,AFG,2020-02-19,0.40,4.000,0.800,5.200,1.400,-1.600
3,Afghanistan,AFG,2020-02-20,0.00,3.833,0.833,5.000,1.000,-0.333
4,Afghanistan,AFG,2020-02-21,0.00,4.571,0.714,5.286,1.429,0.571


In [6]:
covid_data.location.nunique()

230

In [7]:
# covid_data.location.unique().tolist()

In [8]:
continent = ['North America','South America','Africa','Asia','Europe']

In [9]:
# removing continents from data frame
# covid_data[~covid_data.location.isin(continent)]

In [10]:
# covid_data.info

In [17]:
covid_data.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN


In [18]:
# getting data for the desired counties as a data frame
au_data = covid_data[covid_data.location == "Australia"]
usa_data = covid_data[covid_data.location == "United States"]
ind_data = covid_data[covid_data.location == "India"]
chn_data = covid_data[covid_data.location == "China"]
uk_data = covid_data[covid_data.location == "United Kingdom"]
bz_data = covid_data[covid_data.location == "Brazil"]
kr_data = covid_data[covid_data.location == "South Korea"]

In [19]:
# makig a list of the countries data frame and then concat them into a data frame.
frames = [au_data,usa_data,ind_data,chn_data,uk_data,bz_data,kr_data]
country_df = pd.concat(frames)

In [20]:
# making data frame with selected columns
country_df = country_df[["iso_code","location","date","total_cases","new_cases","total_deaths"]]
country_df['month_year'] = pd.to_datetime(country_df['date']).dt.to_period('M')
country_df = country_df.drop(columns=['date'])
country_df.dropna(inplace=True)
country_df = country_df.reset_index(drop=True)
country_df.sort_values(by=['month_year'])
country_df

,iso_code,location,total_cases,new_cases,total_deaths,month_year
0,AUS,Australia,27.0,2.0,1.0,2020-03
1,AUS,Australia,30.0,3.0,1.0,2020-03
2,AUS,Australia,39.0,9.0,1.0,2020-03
3,AUS,Australia,52.0,13.0,2.0,2020-03
4,AUS,Australia,55.0,3.0,2.0,2020-03
...,...,...,...,...,...,...
3324,KOR,South Korea,147874.0,452.0,1985.0,2021-06
3325,KOR,South Korea,148273.0,399.0,1988.0,2021-06
3326,KOR,South Korea,148647.0,374.0,1992.0,2021-06
3327,KOR,South Korea,149191.0,544.0,1993.0,2021-06


In [51]:
country_df.groupby(["month_year"])["location"].nunique()

month_year
2020-01    1
2020-02    3
2020-03    7
2020-04    7
2020-05    7
2020-06    7
2020-07    7
2020-08    7
2020-09    7
2020-10    7
2020-11    7
2020-12    7
2021-01    7
2021-02    7
2021-03    7
2021-04    7
2021-05    7
2021-06    7
Freq: M, Name: location, dtype: int64

In [21]:
# removing rows from month_date column which doesn't have values for all the countries.
complete_country = country_df.loc[(country_df["month_year"]!="2020-01") & (country_df["month_year"]!="2020-02") ] 
complete_country

,iso_code,location,total_cases,new_cases,total_deaths,month_year
0,AUS,Australia,27.0,2.0,1.0,2020-03
1,AUS,Australia,30.0,3.0,1.0,2020-03
2,AUS,Australia,39.0,9.0,1.0,2020-03
3,AUS,Australia,52.0,13.0,2.0,2020-03
4,AUS,Australia,55.0,3.0,2.0,2020-03
...,...,...,...,...,...,...
3324,KOR,South Korea,147874.0,452.0,1985.0,2021-06
3325,KOR,South Korea,148273.0,399.0,1988.0,2021-06
3326,KOR,South Korea,148647.0,374.0,1992.0,2021-06
3327,KOR,South Korea,149191.0,544.0,1993.0,2021-06


In [27]:
country_clean = complete_country.groupby(["location","month_year"])[["total_cases","new_cases","total_deaths"]].sum()
country_clean

total_cases  new_cases  total_deaths
location      month_year                                       
Australia     2020-03     3.567900e+04     4534.0         190.0
              2020-04     1.882060e+05     2207.0        1817.0
              2020-05     2.176250e+05      436.0        3071.0
              2020-06     2.226110e+05      718.0        3075.0
              2020-07     3.617200e+05     9360.0        3996.0
...                                ...        ...           ...
United States 2021-02     7.743205e+08  2402013.0    13632524.0
              2021-03     9.167257e+08  1813675.0    16646637.0
              2021-04     9.446474e+08  1885731.0    16964069.0
              2021-05     1.020165e+09   916746.0    18167522.0
              2021-06     5.344181e+08   230961.0     9569897.0

[112 rows x 3 columns]

In [32]:
# making dataframe for for income for the required countries
au_inc = income_data[income_data.Entity == "Australia"]
usa_inc = income_data[income_data.Entity == "United States"]
ind_inc = income_data[income_data.Entity == "India"]
chn_inc = income_data[income_data.Entity == "China"]
uk_inc = income_data[income_data.Entity == "United Kingdom"]
bz_inc = income_data[income_data.Entity == "Brazil"]
kr_inc = income_data[income_data.Entity == "South Korea"]
# au_inc

In [49]:
# making list of data frames and then contacting them 
frames_inc = [au_inc,usa_inc,ind_inc,chn_inc,uk_inc,bz_inc,kr_inc]
country_inc = pd.concat(frames_inc)
country_inc['month_year'] = pd.to_datetime(country_inc['Day']).dt.to_period('M')
country_inc = country_inc.drop(columns=['Day'])
country_inc.dropna(inplace=True)
country_inc = country_inc.reset_index(drop=True)
country_inc.sort_values(by=['month_year'])
# country_inc
# country_inc.groupby(["month_year"])["Entity"].nunique()

,Entity,Code,income_support,month_year
0,Australia,AUS,0,2020-01
2602,Brazil,BRA,0,2020-01
2601,Brazil,BRA,0,2020-01
2600,Brazil,BRA,0,2020-01
2599,Brazil,BRA,0,2020-01
...,...,...,...,...
519,Australia,AUS,0,2021-06
520,Australia,AUS,0,2021-06
2074,China,CHN,1,2021-06
2076,China,CHN,1,2021-06


In [47]:
# removing rows from month_date column which doesn't have values for all the countries.
complete_country_inc = country_inc.loc[(country_inc["month_year"]!="2021-06")] 
# complete_country_inc

In [45]:
rename_country_inc = complete_country_inc.rename(columns={'Entity': 'location'})
rename_country_inc

,location,Code,income_support,month_year
0,Australia,AUS,0,2020-01
1,Australia,AUS,0,2020-01
2,Australia,AUS,0,2020-01
3,Australia,AUS,0,2020-01
4,Australia,AUS,0,2020-01
...,...,...,...,...
3619,South Korea,KOR,1,2021-05
3620,South Korea,KOR,1,2021-05
3621,South Korea,KOR,1,2021-05
3622,South Korea,KOR,1,2021-05


In [62]:
rename_inc = rename_country_inc.loc[(rename_country_inc["month_year"]!="2020-01") & (rename_country_inc["month_year"]!="2020-02") ] 
rename_inc

pandas.core.frame.DataFrame

In [59]:
final_data = pd.merge(complete_country,rename_inc, how="outer", on=["location", "month_year"]))
final_data

,iso_code,location,total_cases,new_cases,total_deaths,month_year,Code,income_support
0,AUS,Australia,27.0,2.0,1.0,2020-03,AUS,0
1,AUS,Australia,27.0,2.0,1.0,2020-03,AUS,0
2,AUS,Australia,27.0,2.0,1.0,2020-03,AUS,0
3,AUS,Australia,27.0,2.0,1.0,2020-03,AUS,0
4,AUS,Australia,27.0,2.0,1.0,2020-03,AUS,0
...,...,...,...,...,...,...,...,...
96038,KOR,South Korea,140799.0,459.0,1963.0,2021-05,KOR,1
96039,KOR,South Korea,140799.0,459.0,1963.0,2021-05,KOR,1
96040,KOR,South Korea,140799.0,459.0,1963.0,2021-05,KOR,1
96041,KOR,South Korea,140799.0,459.0,1963.0,2021-05,KOR,1


In [189]:
# covid_data.head()
# covid_df = round(covid_data.groupby(["location"])["total_cases"].max(),2)
# # type(covid_df)
# covid_df.dropna(inplace=True)
# covid_df

In [190]:
# afg_data = covid_data[covid_data.location == "Afghanistan"].describe()
# afg_data

In [191]:
afg_data['month_year'] = pd.to_datetime(afg_data['date']).dt.to_period('M')
afg_data.head()

<ipython-input-191-fb8835b80fff>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality,month_year
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,2020-02
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,2020-02
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,2020-02
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,2020-02
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,2020-02


In [192]:
# afg_df = afg_data.groupby(["month_year"])[["location","total_cases","new_cases","month_year"]].max()
# afg_df

In [193]:
# afg_df.reset_index(drop=True, inplace=True)
# afg_df

In [194]:
# total_new_cases= afg_data.groupby(["month_year"])["new_cases"].sum()
# total_new_cases

In [195]:
# total_death= afg_data.groupby(["month_year"])["total_deaths"].sum()
# total_death

In [196]:
# merge_data = pd.DataFrame({"total_death":total_death, "total_new_cases": total_new_cases})
# merge_data.reset_index(drop=False, inplace=True)
# merge_data

In [197]:
# afg_df.reset_index(drop=True, inplace=True)
# afg_df

In [198]:
# month_year = afg_df["month_year"]
# afg_df

In [199]:
# merge_data["month_year"] = month_year
# merge_data.reset_index(drop=True, inplace=True)

In [200]:
# merge_afg = pd.merge(merge_data, afg_df, on = "month_year")
# merge_afg


In [201]:
!pip install plotly.express

In [202]:
merge_afg.dtypes

month_year         period[M]
total_death          float64
total_new_cases      float64
location              object
total_cases          float64
new_cases            float64
month_year_str        object
dtype: object

In [203]:
merge_afg["month_year_str"] = merge_afg.month_year.astype('str')

In [204]:
# # use color as third subset for countries
# import plotly.express as px
# # df = px.data.gapminder().query("continent=='Oceania'")
# fig = px.line(merge_afg, x="month_year_str", y="total_new_cases")
# fig = px.line(merge_afg, x="month_year_str", y="total_death")
# fig.show()

In [205]:
covid_data[covid_data.total_cases == 93288.000000]


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
478,AFG,Asia,Afghanistan,2021-06-16,93288.0,16.0,1319.714,3683.0,0.0,54.0,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
15685,CAN,North America,Canada,2020-06-01,93288.0,809.0,881.286,7952.0,46.0,76.0,...,0.5,105.599,7.37,12.0,16.6,NaN,2.5,82.43,0.929,NaN


In [206]:
len(covid_data)

96208

In [207]:
debt_data.head()

,Entity,Code,Day,debt_relief
0,Afghanistan,AFG,2020-01-01,0
1,Afghanistan,AFG,2020-01-02,0
2,Afghanistan,AFG,2020-01-03,0
3,Afghanistan,AFG,2020-01-04,0
4,Afghanistan,AFG,2020-01-05,0


In [208]:
income_data.head()


,Entity,Code,Day,income_support
0,Afghanistan,AFG,2020-01-01,0
1,Afghanistan,AFG,2020-01-02,0
2,Afghanistan,AFG,2020-01-03,0
3,Afghanistan,AFG,2020-01-04,0
4,Afghanistan,AFG,2020-01-05,0


In [209]:
income_afg = income_data[income_data.Entity == "Afghanistan"]
income_afg

,Entity,Code,Day,income_support
0,Afghanistan,AFG,2020-01-01,0
1,Afghanistan,AFG,2020-01-02,0
2,Afghanistan,AFG,2020-01-03,0
3,Afghanistan,AFG,2020-01-04,0
4,Afghanistan,AFG,2020-01-05,0
...,...,...,...,...
508,Afghanistan,AFG,2021-06-09,0
509,Afghanistan,AFG,2021-06-10,0
510,Afghanistan,AFG,2021-06-11,0
511,Afghanistan,AFG,2021-06-12,0


In [210]:
income_afg['month_year'] = pd.to_datetime(income_afg['Day']).dt.to_period('M')
income_afg.head()

<ipython-input-210-c8bc48e76437>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Entity,Code,Day,income_support,month_year
0,Afghanistan,AFG,2020-01-01,0,2020-01
1,Afghanistan,AFG,2020-01-02,0,2020-01
2,Afghanistan,AFG,2020-01-03,0,2020-01
3,Afghanistan,AFG,2020-01-04,0,2020-01
4,Afghanistan,AFG,2020-01-05,0,2020-01


In [211]:
income_df = income_afg.groupby(["month_year"])[["Entity","income_support","month_year"]].max()

income_df.reset_index(drop=True, inplace=True)
income_df

,Entity,income_support,month_year
0,Afghanistan,0,2020-01
1,Afghanistan,0,2020-02
2,Afghanistan,0,2020-03
3,Afghanistan,0,2020-04
4,Afghanistan,0,2020-05
5,Afghanistan,0,2020-06
6,Afghanistan,0,2020-07
7,Afghanistan,0,2020-08
8,Afghanistan,0,2020-09
9,Afghanistan,0,2020-10


In [212]:
agf_2_income = pd.merge(merge_afg,income_df, on = "month_year")
agf_2_income.drop(['Entity','month_year'], axis=1)

,total_death,total_new_cases,location,total_cases,new_cases,month_year_str,income_support
0,0.0,1.0,Afghanistan,1.0,1.0,2020-02,0
1,27.0,174.0,Afghanistan,175.0,33.0,2020-03,0
2,890.0,1952.0,Afghanistan,2127.0,187.0,2020-04,0
3,5007.0,13081.0,Afghanistan,15208.0,866.0,2020-05,0
4,15016.0,16299.0,Afghanistan,31507.0,915.0,2020-06,0
5,33518.0,5158.0,Afghanistan,36665.0,348.0,2020-07,0
6,42101.0,1494.0,Afghanistan,38159.0,108.0,2020-08,0
7,42987.0,1109.0,Afghanistan,39268.0,125.0,2020-09,0
8,46287.0,2157.0,Afghanistan,41425.0,141.0,2020-10,0
9,49133.0,4849.0,Afghanistan,46274.0,282.0,2020-11,0


In [213]:
# income_afg["month_year_str"] = income_afg.month_year.astype('str')

In [214]:
# # use color as third subset for countries
# import plotly.express as px
# # df = px.data.gapminder().query("continent=='Oceania'")
# fig = px.line(merge_afg, x="month_year_str", y="total_new_cases")
# fig.show()

In [215]:

# income_df = round(income_data.groupby(["Entity"])["income_support"].mean(),2)
# income_df
# # income_df.sort_values(ascending = False)

In [216]:
# df = income_data.set_index("Entity")

# # df = income_data.loc["Afghanistan"]
# df_income = df.loc["Bulgaria"]
# df_income["income_support"].sort_values(ascending = False)

In [217]:
# face_data.head()

In [218]:
# travel_data.head()

In [219]:
# school_data.head()
